In [112]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
import numpy as np 
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
import scipy.stats

In [113]:
df = pd.read_csv('diabetes.csv')
df.shape

(768, 9)

In [114]:
X = df.drop(columns = 'Outcome')
y = df['Outcome']

In [115]:
scaler = StandardScaler()
scaler.fit(X)
X_transformed = scaler.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size = 0.1, stratify = y, random_state=42)

In [116]:
model = svm.SVC(kernel="linear", probability=True)
model.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

In [117]:
acc_score = accuracy_score(model.predict(X_train), y_train)
print(f"Accuracy score for train data is {acc_score}")
acc_score = accuracy_score(model.predict(X_test), y_test)
print(f"Accuracy score for test data is {acc_score}")

Accuracy score for train data is 0.7771345875542692
Accuracy score for test data is 0.7402597402597403


In [118]:
# to numpy array
X = X_train
Y = y_train
classifier = model # or regressor

In [119]:
def plot_probability_surface(classifier, levels=20, points=True):
    xx, yy = np.meshgrid(np.linspace(5, 30, 500), np.linspace(5, 45, 500))
    Z = classifier.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    Z = Z.reshape(xx.shape)
    fig, axis = plt.subplots(1, 1, figsize=(10, 10))
    axis.contourf(xx, yy, Z, alpha=0.75, cmap='bone', vmin=0, vmax=1, levels=levels)
    if points:
        axis.scatter(X[:, 0], X[:, 1], c=Y, s=15,
                    cmap='bone', edgecolors='black', linewidth=.5)
    axis.axis('off')
    return axis
# ax = plot_probability_surface(classifier) # cant plot for 8 features atm

In [120]:
def cost_function(x_prime, x, y_prime, lambda_value, model, X):
    mad =  scipy.stats.median_abs_deviation(X, axis=0)
    distance = np.sum(np.abs(x-x_prime)/mad)
    misfit = (model(x_prime, y_prime)-y_prime)**2
    return lambda_value * misfit + distance

In [121]:
def evaluate_model(x, y_prime):
    # round the y_prime value to provide the right class [0,1]
    # predicted_prob = classifier.predict_proba(x.reshape((1, -1)))[0,int(np.round(y_prime))]
    predicted_prob = classifier.predict_proba(x.reshape((1, -1)))[0][0]
    #print(predicted_prob)
    round_res = predicted_prob = classifier.predict_proba(x.reshape((1, -1)))[0,int(np.round(y_prime))]
    print(f'y_prime: {y_prime}, predicted_prob: {predicted_prob}, round_res: {round_res}')
    return predicted_prob

In [122]:
def get_counterfactuals(x, y_prime_target, model):
    eps = .05 # tolerance
    lambda_min =  1e-10
    lambda_max = 1e5
    lambda_steps = 30
    lambdas = np.logspace(np.log10(lambda_min), 
                            np.log10(lambda_max), 
                            lambda_steps)
    # scan over lambda
    candidates = []
    Y_primes = []
    for lambda_k in lambdas:
        arguments = x, y_prime_target, lambda_k, model, X
        # optimise the cost function -- assuming here it's smooth
        solution = scipy.optimize.minimize(cost_function, 
                                           x, # start from our current observation
                                           args=arguments)
        x_prime_hat = solution.x
        Y_primes.append(model(x_prime_hat, y_prime_target))
        candidates.append(x_prime_hat)
    Y_primes = np.array(Y_primes)
    candidates = np.array(candidates)
    # check if any counterfactual candidates meet the tolerance condition
    eps_condition = np.abs(Y_primes - y_prime_target) <= eps
    # plot y_primes over lambda
    #print(Y_primes)
    # plot y_primes over lambda. use log scale for lambda
    plt.plot(np.log10(lambdas), Y_primes)
    plt.show()
    return candidates[eps_condition]

In [123]:
instance = 1
print(f'instance {instance} has features {X[instance]} and label {Y[instance]}')
print('model prediction: ', classifier.predict(X[instance].reshape(1, -1)))
print('model probability: ', classifier.predict_proba(X[instance].reshape(1, -1)))

instance 1 has features [-0.84488505 -1.02950492 -2.02166474  1.34636635  0.16671612  2.92008462
  0.07285643 -0.61611067] and label 0
model prediction:  [0]
model probability:  [[0.51650423 0.48349577]]


In [124]:
P_prime = 0.1
counterfactuals = get_counterfactuals(X[instance], P_prime, evaluate_model)
counterfactuals

y_prime: 0.1, predicted_prob: 0.5165042284236109, round_res: 0.5165042284236109
y_prime: 0.1, predicted_prob: 0.5165042271052652, round_res: 0.5165042271052652
y_prime: 0.1, predicted_prob: 0.516504224422801, round_res: 0.516504224422801
y_prime: 0.1, predicted_prob: 0.5165042292293752, round_res: 0.5165042292293752
y_prime: 0.1, predicted_prob: 0.5165042288524074, round_res: 0.5165042288524074
y_prime: 0.1, predicted_prob: 0.5165042284665066, round_res: 0.5165042284665066
y_prime: 0.1, predicted_prob: 0.5165042260194092, round_res: 0.5165042260194092
y_prime: 0.1, predicted_prob: 0.5165042274947258, round_res: 0.5165042274947258
y_prime: 0.1, predicted_prob: 0.5165042281628685, round_res: 0.5165042281628685
y_prime: 0.1, predicted_prob: 0.6584036811546373, round_res: 0.6584036811546373
y_prime: 0.1, predicted_prob: 0.6584036800349328, round_res: 0.6584036800349328
y_prime: 0.1, predicted_prob: 0.6584036777565934, round_res: 0.6584036777565934
y_prime: 0.1, predicted_prob: 0.6584036818

In [ ]:
best_counterfactual = counterfactuals[-1]
print("best counterfactual:", best_counterfactual)
print("model prediction on counterfactual:", classifier.predict(best_counterfactual.reshape(1, -1)))
print("Model prediction probs on counterfactual:", classifier.predict_proba(best_counterfactual.reshape(1, -1)))
print("% difference (x', x):", (100*(best_counterfactual-X[instance])/X[instance]).round(2))

best counterfactual: [-0.84488506  1.06933744 -2.02166475  1.34636635  0.16671612  2.92008462
  0.07285642 -0.61611067]
model prediction on counterfactual: [1]
Model prediction probs on counterfactual: [[0.10008193 0.89991807]]
% difference (x', x): [   0.   -203.87    0.     -0.     -0.     -0.     -0.      0.  ]
